In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
import numpy as np

from PreprocessingKeyWords import preprocessing
preprocessor=preprocessing()

from Word2Vec import Word2Vec2
word2vecmodel=Word2Vec2()
model=word2vecmodel.loadWord2Vec()

[[-0.1076434   0.05424118 -0.00710406 ... -0.01430076 -0.06561279
   0.07780457]
 [-0.14706644  0.08422852 -0.02103778 ... -0.03216404 -0.08666397
   0.08807224]
 [-0.14288445  0.09338379  0.00132895 ... -0.02467202 -0.08329341
   0.08229497]
 ...
 [-0.1116906   0.08510418 -0.01746727 ... -0.0069879  -0.09046663
   0.09317452]
 [-0.12336561  0.08139716 -0.0101454  ... -0.01888311 -0.09106445
   0.08679199]
 [-0.13280472  0.08436351 -0.01240976 ... -0.02877154 -0.08762718
   0.11678299]]




 [[ 0.06539663  0.06683604  0.05779203 ... -0.0880839   0.02461243
  -0.03683599]
 [ 0.04991998  0.02516831  0.0042589  ... -0.08509657  0.07657877
  -0.04275174]
 [ 0.0348053   0.00196966  0.07784017 ... -0.03153992  0.01524353
  -0.01417033]
 ...
 [ 0.03589545  0.03853353  0.0612793  ... -0.02773709  0.04360623
  -0.07706705]
 [ 0.05375126  0.03165763  0.0498788  ... -0.09249878  0.03982816
  -0.02919224]
 [-0.01140698  0.03577238  0.02974722 ... -0.01321494  0.0154386
  -0.05046123]]


In [2]:
train_data=pd.read_csv("trainData.csv")

#Data Preprocessing

#removing the hmid
train_data = train_data.drop(['hmid','concepts'], axis=1)

#remove missing values
train_data=train_data.replace(('prefer not to say'),np.NAN)
#print(train_data.isnull().sum())
train_data=train_data.dropna()

#print(train_data.isnull().sum())
# print(len(train_data))

#creating features for moments data
#happycorpus=preprocessor.getSocialKeywords(train_data['moment'])
happycorpus=preprocessor.getAgencyKeywords(train_data['moment'])
happycorpus=preprocessor.getSocialNGrams(train_data['moment'])

featureMatrix=word2vecmodel.getFeatureMatrix_Ngrams(model,happycorpus)

In [6]:
y_agency = train_data.agency
y_social=train_data.social
print("Step 1:",type(y_social))

#train n test data split
train_X, test_X, train_y, test_y = train_test_split(featureMatrix, y_social, test_size=0.25)
print("done !!")

# train_X, test_X, train_y, test_y = train_test_split(featureMatrix, y_agency, test_size=0.25)
# print("done !!")


Step 1: <class 'pandas.core.series.Series'>
done !!


In [7]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(train_X, train_y)
print(model)
# make predictions for test data
y_pred = model.predict(test_X)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [8]:
print(y_pred)
predictions = [value for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

['yes' 'no' 'no' ... 'no' 'yes' 'no']
Accuracy: 84.59%
